In [1]:
import sys 
import os 
import time 
import re
import datetime
import pandas as pd 
import numpy as np 
from collections import Counter
import pandas_profiling

# 11月数据

In [2]:
def df_Nov():  
    df1 = pd.read_excel(os.path.join(file_path_in, 'PC LongNumber Nov 2019(2).xlsx'))
    df2 = pd.read_excel(os.path.join(file_path_in, 'PC OTP Nov 2019.xlsx'))
    df = pd.concat([df1, df2], axis = 0)
    return df

# 10月数据

In [3]:
def df_Oct():
    df1 = pd.read_excel(os.path.join(file_path_in, 'PC LongNumber Oct 2019(2).xlsx'))
    df2 = pd.read_excel(os.path.join(file_path_in, 'PC OTP Oct 2019(2).xlsx'))
    df = pd.concat([df1, df2], axis = 0)
    return df

# 9月数据

In [4]:
def df_Sept():
    df1 = pd.read_excel(os.path.join(file_path_in, 'PC LongNumber Sept 2019.xlsx'))
    df2 = pd.read_excel(os.path.join(file_path_in, 'PC OTP Sept 2019.xlsx'))
    df3 = pd.read_excel(os.path.join(file_path_in, 'PC Non-OTP Sept 2019.xlsx'))
    df = pd.concat([df1, df2, df3], axis = 0)
    df = df[df['Operator'] != 'Overseas']
    return df

# ALL

In [5]:
def df_all():
    df1 = pd.read_excel(os.path.join(file_path_in, 'PC LongNumber Nov 2019(2).xlsx'))
    df2 = pd.read_excel(os.path.join(file_path_in, 'PC OTP Nov 2019.xlsx'))
    df3 = pd.read_excel(os.path.join(file_path_in, 'PC LongNumber Oct 2019(2).xlsx'))
    df4 = pd.read_excel(os.path.join(file_path_in, 'PC OTP Oct 2019(2).xlsx'))
    df5 = pd.read_excel(os.path.join(file_path_in, 'PC LongNumber Sept 2019.xlsx'))
    df6 = pd.read_excel(os.path.join(file_path_in, 'PC OTP Sept 2019.xlsx'))
    df7 = pd.read_excel(os.path.join(file_path_in, 'PC Non-OTP Sept 2019.xlsx'))
    df8 = pd.concat([df5, df6, df7], axis = 0)
    df8 = df8[df8['Operator'] != 'Overseas']
    df = pd.concat([df1, df2,df3, df4,df8], axis = 0)
    return df

# 分析数据

In [6]:
file_path_in = '/Users/yantingting/Documents/海外风控/印尼/MGI数据测试/all_data/'
file_path_out = '/Users/yantingting/Documents/海外风控/印尼/MGI数据测试/my_analysis/Nov'
if not os.path.exists(file_path_out):
    os.makedirs(file_path_out)

In [7]:
df = df_Nov()
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1330932 entries, 0 to 303242
Data columns (total 9 columns):
Message ID         1330932 non-null int64
Date/Time          1330932 non-null datetime64[ns]
From               1330932 non-null object
Send to            1330932 non-null int64
Message Title      1330932 non-null object
Operator           1330932 non-null object
Message Status     1330932 non-null object
No. of SMS         1330932 non-null int64
Message Content    1330932 non-null object
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 101.5+ MB


,Message ID,Date/Time,From,Send to,Message Title,Operator,Message Status,No. of SMS,Message Content
0,21191898194,2019-11-30 23:59:08,CMK,628156961290,Indosat,Indonesia,DELIVERED,1,(Tunai Cepat) Pelunasan Anda Rp 1400000berhasi...
1,21191898028,2019-11-30 23:58:26,CMK,6281395300667,Telkomsel,Indonesia,DELIVERED,1,(Tunai Cepat) Pelunasan Anda Rp 1200000berhasi...
2,21191897974,2019-11-30 23:58:13,CMK,6285933192866,XL,Indonesia,DELIVERED,1,(Tunai Cepat)Perpanjangan Anda Rp 307000 berha...
3,21191897775,2019-11-30 23:57:24,CMK,6282350758802,Telkomsel,Indonesia,DELIVERED,1,(Dana Tercepat) Perpanjangan Anda sebesar Rp 2...
4,23198282931,2019-11-30 23:55:14,CMK,6281553106506,Indosat,Indonesia,DELIVERED,1,(Dana Tercepat) Perpanjangan Anda sebesar Rp 3...


In [8]:
# 匹配APP 和短信内容
def re_fun1(x, n):
    try:
       mm = re.split(r'\(|\[|\]|\)', x)[n] 
    except IndexError:
       mm = None 
    return mm

# 匹配 金额
def re_fun2(x):
    try:
       mm = re.search('rp(.)(\d+)', x, re.I).group()
    except AttributeError:
       mm = None
    return mm
# 匹配验证码短信 (开头是数字)
def re_fun3(x):
    try:
       mm = re.match('(\d+)(.)', x, re.I).group()
    except AttributeError:
       mm = None
    return mm
# 银行账号
def re_fun4(x):
    try:
       mm = re.search('belakang (\d+)', x, re.I).group()
    except AttributeError:
       mm = None
    return mm
# 提醒的还款人
def re_fun5(x):
    try:
       mm = re.search('mengingatkan(.*)melakukan', x, re.I).group()
    except AttributeError:
       mm = None
    return mm

# 向借款人发送还款信息提醒
def re_fun6(x):
    try:
       mm = re.search('kpd(.*)pembayaran', x, re.I).group()
    except AttributeError:
       mm = None
    return mm        

# 匹配日期
def re_fun7(x):
    try:
       mm = re.search('(\d+)(-)(\d+)(-)(\d+)', x, re.I).group()
    except AttributeError:
       mm = None
    return mm 

# 匹配连接地址
def re_fun8(x):
    try:
       mm = re.search('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', x, re.I).group()
    except AttributeError:
       mm = None
    return mm 
# 匹配截止日期前的字段
def re_fun9(x):
    try:
       mm = re.search('(Tgl jatuh tempo)|(Waktu jatuh tempo)|(Tanggal jatuh tempo)', x, re.I).group()
    except AttributeError:
       mm = None
    return mm 

# # 逾期n天提醒
# def re_fun10(x):
#     try:
#        mm = re.search('Tagihanmu(.*)amount', x, re.I).group()
#     except AttributeError:
#        mm = None
#     return mm 
# 逾期n天提醒
def re_fun11(x):
    try:
       mm = re.search('(\d+)(.)hari', x, re.I).group()
    except AttributeError:
       mm = None
    return mm 

In [9]:
df['app_name'] = df['Message Content'].apply(lambda x: re_fun1(x, 1))
df['content'] = df.apply(lambda x: x['Message Content'].replace(x['app_name'], '') if x['app_name'] is not None else x['Message Content'], axis = 1)
df['content'] = df['content'].apply(lambda x: x.replace('(','').replace('[', '').replace(']','').replace(')','').strip())
df['app_name'] = df['app_name'].apply(lambda x: x.replace('Wall-in', 'Wall In').replace('wall in', 'Wall In').replace('Pinjam uang','Pinjam Uang') if x is not None else x)
df['amount'] = df['content'].apply(lambda x: re_fun2(x))
df['message_type'] = df['content'].apply(lambda x: 'yzm' if re_fun3(x) else 'other')
df['remind_sb'] = df['content'].apply(lambda x: re_fun5(x))
df['remind_self'] = df['content'].apply(lambda x: re_fun6(x))
df['bank_code'] = df['content'].apply(lambda x: re_fun4(x))
df['deadline'] = df['content'].apply(lambda x: re_fun7(x))
df['click'] = df['content'].apply(lambda x: re_fun8(x))
df['content'] = df['content'].apply(lambda x: x.replace(re_fun2(x), 'amount') if re_fun2(x) is not None else x)                             
df['content'] = df['content'].apply(lambda x: x.replace(re_fun3(x), '') if re_fun3(x) is not None else x)
df['content'] = df['content'].apply(lambda x: x.replace(re_fun5(x), 'remind_sb') if re_fun5(x) is not None else x)
df['content'] = df['content'].apply(lambda x: x.replace(re_fun4(x), 'bank_code') if re_fun4(x) is not None else x)
df['content'] = df['content'].apply(lambda x: x.replace(re_fun7(x), 'deadline') if re_fun7(x) is not None else x)
df['content'] = df['content'].apply(lambda x: x.replace(re_fun6(x), 'remind_self') if re_fun6(x) is not None else x )
df['content'] = df['content'].apply(lambda x: x.replace(re_fun8(x), 'click').replace(':', '') if re_fun8(x) is not None else x )
df['content'] = df['content'].apply(lambda x: x.replace(re_fun9(x), '截止日期') if re_fun9(x) is not None else x )
df['dpd_n'] = df['content'].apply(lambda x: re_fun11(x))
df['content'] = df['content'].apply(lambda x: x.replace(re_fun11(x), '账单已逾期N天') if re_fun11(x) is not None else x )



In [10]:
message_dict ={'Pelunasan Anda amountberhasil. Anda BERKESEMPATAN utk naik limit dan potongan bunga dipengajuan berikutnya click': '还款成功',
         'Perpanjangan Anda amount berhasil. 截止日期 sudah berubah menjadi deadline.Cek aplikasiclick': '展期成功',
         'Perpanjangan Anda sebesar amount berhasil. Jatuh tempo telah berubah menjadi deadline. Cek aplikasi  click':'展期成功',
         'Info Tagihan amount sudah lunas, nikmati potongan bunga dan naik limit dgn pengajuan kembali click': '还款成功',
         'Pembayaran sebesar amount. sdh lunas. Dapatkan KESEMPATAN naik limit dan potongan bunga dgn pengajuan kembali click': '还款成功',
         'Selamat!Perpanjangan amount sudah berhasil. 截止日期 menjadi deadline.Login aplikasi click': '展期成功',
         'Pelunasan Anda amount berhasil. Dapatin kesempatan potongan bunga dan naik limit dipengajuan berikutnya click': '还款成功',
         'Pelunasan amountsudah dibayarkan. Nikmati potongan bunga dan kenaikan limit dengan pengajuan kembali click': '还款成功',
         'Pembayaran perpanjanganmu amount sudah berhasil. 截止日期 sudah berubah menjadi deadline.click': '展期成功',
         'Terima kasih! Tagihanmu amountsudah dilunasi. Raih kesempatan naik limit dgn ajukan kembali click': '还款成功',
         'Terima kasih atas pelunasan tagihan Anda sebesar amount. Yuk login app untuk melakukan pengajuan kembali  click': '还款失败请重新提交',
         '截止日期 pengajuan Anda sebesar amount berhasil diperbarui menjadi  deadline. Yuk login app untuk detailnya  click':'展期成功',
         'Anda berhasil melunaskan amount. dapatkan kesempatan naik limit dan potongan bunga dipengajuan berikutnya click': '还款成功',
         'Pinjaman Anda amount sudah lunas, nikmati potongan bunga dan kenaikan limit dipengajuan selanjutnya click': '还款成功',
         'Hari ini 截止日期 tagihanmu, sebesar amount di . Login app  click utk melakukan pelunasan/ perpanjangan.': '账单今日到期请还款',
         'Tagihan Anda sebesar amount telah jatuh tempo hari ini. Silahkan login app click untuk melakukan pelunasan/ perpanjangan': '账单今日到期请还款',
         'Tagihan Anda sebesar amount telah jatuh tempo hari ini. Silahkan login app  click untuk melakukan pelunasan/ perpanjangan': '账单今日到期请还款',
         'Tagihanmu di  jatuh tempo hari ini sebesar amount. Login app click utk melakukan pembayaran.': '账单今日到期请还款',
         'Hai~ dgn pembayaran lebih awal atau tepat waktu akan berkesempatan dapatkan potongan bunga dan naik limit': '提早或及时付款，您将有机会获得贴现利息并提高限额',
         'Tagihanmu jatuh tempo hari ini dgn nominal amount.Login aplikasi utk pelunasan atau perpanjangan click': '账单今日到期请还款',
         'Yth Bapak/Ibu, Kami dari  memohon bantuannya utk remind_sb pelunasan/perpanjangan. Keterlambatan akan menyebabkan denda terus bertambah. Terima kasih': '提醒某人还款',
         'Yth Bapak/Ibu, kami memohon bantuan Anda utk segera remind_sb pelunasan/perpanjangan. Keterlambatan akan menyebabkan denda terus bertambah. Terima kasih': '提醒某人还款',
         'Tagihan Anda telah terlambat dilunasi 账单已逾期N天. Jumlah tagihan saat ini amount. Keterlambatan telah dikenakan denda. Silahkan segera login app click jika perlu melakukan perpanjangan atau pelunasan': '账单已逾期N天请还款',
         'Tagihan Anda telah terlambat dilunasi 账单已逾期N天. Jumlah tagihan saat ini amount. Demi menghindari denda terus bertambah, harap segera login app click untuk melakukan perpanjangan atau pelunasan': '账单已逾期N天请还款',
         'Tagihan Anda telah terlambat dilunasi 账单已逾期N天. Jumlah tagihan saat ini amount. Denda keterlambatan terus bertambah. Mohon segera login app  click atau hubungi kami utk melakukan pelunasan/ perpanjangan agar denda tidak terus bertambah': '账单已逾期N天请还款',
         'Telah terlambat 账单已逾期N天 tagihanmu di . Jumlah tagihan saat ini amount. Keterlambatan dikenakan denda. Silahkan segera login app click untuk melakukan perpanjangan/pelunasan': '账单已逾期N天请还款',
         'Telah terlambat 账单已逾期N天 tagihanmu di .Jumlah tagihan saat ini amount. Demi menghindari denda terus bertambah, segera login app click utk melakukan perpanjangan/pelunasan': '账单已逾期N天请还款',
         'Telah terlambat 账单已逾期N天 tagihanmu di . Jumlah tagihan saat ini amount. Denda terus bertambah, Segera login app  click atau hubungi kami utk melakukan pelunasan/perpanjangan.': '账单已逾期N天请还款',
         'Yth Bapak/Ibu, kami memohon bantuan Anda utk segera remind_sb pelunasan/ perpanjangan. Keterlambatan berkelanjutan akan menyebabkan denda terus bertambah. Terima kasih': '提醒某人还款',
         'Bpk/ibu yg terhormat, Kami dari Sumber Dana. Mohon sampaikan pesan remind_self agar denda tidak terus bertambah. Terima Kasih': '账单已逾期N天请还款',
         'Yth Bapak/Ibu, Kami dari pihak  memohon bantuan utk remind_sb pembayaran. Agar keterlambatan tidak terus bertambah. Terima Kasih.': '提醒某人还款',
         'Tagihanmu di  telah terlambat 账单已逾期N天 dengan jumlah tagihan saat ini sebesar amount. Keterlambatan akan dikenakan denda. Segera login app click untuk melakukan pembayaran': '账单已逾期N天请还款',
         'Tagihanmu di  sudah telat 账单已逾期N天 dgn pelunasan saat ini amount.Keterlambatan akan dikenakan denda. Login aplikasi click': '账单已逾期N天请还款',
         'Pengumuman: Anda berhasil menggunakan 2 kupon dan Anda hemat amount': 'other',
         'Pemberitahuan! Telah dilunaskan pinjaman amount.Dapatkan potongan bunga dan kenaikan limit dipengajuan berikut.click': 'other',
         'Pengumuman: Anda berhasil menggunakan 1 kupon dan Anda hemat amount': 'other',
         'Pemberitahuan! Pembayaran periode cicilan ke-1 Anda sudah berhasil.click': 'other',
         'Pelunasan amount dan pembelian kupon sudah berhasil. Dapatkan potongan bunga dan kenaikan limit dipengajuan berikut.click': 'other',
         'Pemberitahuan! Pembayaran periode cicilan ke-3 Anda sudah berhasil.click': 'other',
         'Pemberitahuan! Pembayaran periode cicilan ke-2 dan pembelian kupon Anda sudah berhasil.click': 'other',
         'Pengumuman: Anda berhasil menggunakan 3 kupon dan Anda hemat amount': 'other',
         'Pengumuman Anda berhasil menggunakan 2 kupon dan Anda hemat amountclick': 'other',
         'Pemberitahuan! Telah dilunaskan pinjaman amountDapatkan potongan bunga dan kenaikan limit dipengajuan berikut.click': 'other',
         'Pengumuman Anda berhasil menggunakan 1 kupon dan Anda hemat amountclick': 'other',
         'Pemberitahuan! Pembayaran periode cicilan ke-1 dan pembelian kupon Anda sudah berhasil.click': 'other',
         'Pemberitahuan! Pembayaran periode cicilan ke-1 dan ke-2 Anda sudah berhasil.click': 'other',
         'Cicilan kedua-mu sudah telat 账单已逾期N天. Pelunasan saat ini amount. Keterlambatan akan dikenakan denda. Login aplikasi click': 'other',
         ' Masa berlaku: 90 detik. Apabila gagal silahkan ulangi kembali': '验证码',
         'Dana sebesar amount telah dicairkan ke rekening bank Anda yang bernomor bank_code. Pengajuan Anda akan jatuh tempo pada deadline': '资金已到账',
         'Telah berhasil dicairkan amount ke rekening bank mu yang bernomor bank_code.截止日期:deadline': '资金已到账',
         'Dana sebesar amounttelah berhasil dicairkan ke rekening bank Anda yang bernomor bank_code. 截止日期: deadline': '资金已到账',
         ' Masa berlaku: 90 detik. Apabila gagal silakan ulangi kembali': '验证码',
         'Pinjaman Anda sudah berhasil Dicairkan ke rekening Anda yang bernomor bank_code. 截止日期 cicilan pertama:deadline': '资金已到账',
         'Pinjaman Anda sudah berhasil Dicairkan ke rekening Anda yang bernomor bank_code. 截止日期:deadline': '资金已到账',
         'Telah berhasil Dicairkan ke rekening Anda yg bernomor bank_code. 截止日期:deadlinePembelian kupon berhasil.': '资金已到账',
         'Telah berhasil Dicairkan ke rekening Anda yg bernomor bank_code. 截止日期 cicilan pertama:deadlinePembelian kupon berhasil.': '资金已到账',
         'Yg terhormat bpk/ibu, Kami dari One Hope. Mohon sampaikan pesan remind_self agar denda tidak terus bertambah. Terima Kasih': 'other',
         'Tagihanmu di  sudah telat 账单已逾期N天 dgn pelunasan saat ini amount .Keterlambatan akan dikenakan denda. Login aplikasi click': 'other',
         'Pembayaran perpanjanganmu amount  sudah berhasil. 截止日期 sudah berubah menjadi deadline .click': 'other',
         'Terima kasih! Tagihanmu amount sudah dilunasi. Raih kesempatan naik limit dgn ajukan kembali click': 'other',
         'Tagihanmu jatuh tempo hari ini dgn nominal amount .Login aplikasi utk pelunasan atau perpanjangan click': 'other',
         'Dana sebesar amount telah berhasil dicairkan ke rekening bank Anda yang bernomor bank_code. 截止日期: deadline': '资金已到账',
         'Selesaikan informasi data dan pengajuan maka Anda berkesempatan nikmati potongan bunga dan pinjaman limit tinggi. click': 'other'}


df['message'] = df['content'].apply(lambda x: message_dict.get(x))

In [31]:
use_cols = ['Message ID', 'Date/Time', 'From', 'Send to', 'Message Title','Message Status',
            'app_name','amount','bank_code', 'message_type', 'deadline','remind_sb', 'remind_self', 
            'message', 'dpd_n']
my_data = df[use_cols]
my_data.to_csv(os.path.join(file_path_out, 'my_data.csv'), index = False)

In [3]:
my_data= pd.read_csv(os.path.join(file_path_out, 'my_data.csv'))
my_data['deadline'] = my_data['deadline'].apply(lambda x: x if isinstance(x, float) else re.split('-',x) )
my_data['deadline'] = my_data['deadline'].apply(lambda x: x if isinstance(x, float) else
                                                '-'.join([x[2], x[1], x[0]]) if x[2] == '2019' else
                                                '-'.join([x[0], x[1], x[2]]))
my_data['deadline'] = my_data['deadline'].apply(lambda x: x if isinstance(x, float) else datetime.datetime.strptime(x, '%Y-%m-%d'))
my_data['Date/Time'] = my_data['Date/Time'].apply(lambda x: x if isinstance(x, float) else datetime.datetime.strptime(x[:10], '%Y-%m-%d'))
print(1)
my_data['period'] = my_data.apply(lambda x :  x if isinstance(x['deadline'], float) else (x['deadline'] - x['Date/Time']).days, axis = 1)
# my_data['period'] = my_data.apply(lambda x : x if isinstance(x['deadline'], float) else 
#                                   datetime.datetime.strptime(x['deadline'], '%Y-%m-%d') - datetime.datetime.strptime(x['Date/Time'][:10], '%Y-%m-%d'), axis = 1)
my_data.info()
my_data.head()
my_data.to_csv(file_path_out + 'my_data1.csv', index = False)

/Users/yantingting/anaconda3/envs/modeling/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8,11,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1330932 entries, 0 to 1330931
Data columns (total 16 columns):
Message ID        1330932 non-null int64
Date/Time         1330932 non-null datetime64[ns]
From              1330932 non-null object
Send to           1330932 non-null int64
Message Title     1330932 non-null object
Message Status    1330932 non-null object
app_name          1314323 non-null object
amount            800632 non-null object
bank_code         104905 non-null object
message_type      1330932 non-null object
deadline          159440 non-null datetime64[ns]
remind_sb         309148 non-null object
remind_self       16608 non-null object
message           1330932 non-null object
dpd_n             272158 non-null object
period            159440 non-null float64
dtypes: datetime64[ns](2), float64(1), int64(2), object(11)
memory usage: 162.5+ MB


,Message ID,Date/Time,From,Send to,Message Title,Message Status,app_name,amount,bank_code,message_type,deadline,remind_sb,remind_self,message,dpd_n,period
0,21191898194,2019-11-30,CMK,628156961290,Indosat,DELIVERED,Tunai Cepat,Rp 1400000,NaN,other,NaT,NaN,NaN,还款成功,NaN,NaN
1,21191898028,2019-11-30,CMK,6281395300667,Telkomsel,DELIVERED,Tunai Cepat,Rp 1200000,NaN,other,NaT,NaN,NaN,还款成功,NaN,NaN
2,21191897974,2019-11-30,CMK,6285933192866,XL,DELIVERED,Tunai Cepat,Rp 307000,NaN,other,2019-12-08,NaN,NaN,展期成功,NaN,8.0
3,21191897775,2019-11-30,CMK,6282350758802,Telkomsel,DELIVERED,Dana Tercepat,Rp 263000,NaN,other,2019-12-08,NaN,NaN,展期成功,NaN,8.0
4,23198282931,2019-11-30,CMK,6281553106506,Indosat,DELIVERED,Dana Tercepat,Rp 367000,NaN,other,2019-12-08,NaN,NaN,展期成功,NaN,8.0


# 读取处理好的数据并进行变量衍生

In [3]:
my_data = pd.read_csv(file_path_out + 'my_data1.csv')

/Users/yantingting/anaconda3/envs/modeling/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8,11,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
app_dict = {'Tunai Cepat': 'Tunai Cepat',
         'Dana Tercepat': 'Dana Tercepat',
         'Sumber Dana': 'Sumber Dana',
         'Pinjam Uang': 'Pinjam Uang',
         'Wall In': 'Wall In',
         'Dana Segera': 'Dana Segera',
         '': 'other',
         'KoinMu': 'other',
         'Pasar Uang': 'other',
         'One Hope': 'other',
         'Modal Pinjaman': 'other',
         'Lahan Uang': 'other'}
my_data['app_name1'] = my_data['app_name'].apply(lambda x: app_dict.get(x))
my_data['amount'] = my_data['amount'].apply(lambda x: np.nan if isinstance(x,float) else re.search('\d+', x).group())
my_data['amount'] = my_data['amount'].astype('float64')
my_data['cur_fund'] = my_data.apply(lambda x :(datetime.datetime.strptime('2019-12-18', '%Y-%m-%d') - datetime.datetime.strptime(x['Date/Time'], '%Y-%m-%d')).days if x['message'] == '资金已到账' else np.nan, axis = 1)

In [5]:
my_data[['app_name1', 'amount']].head(100)

,app_name1,amount
0,Tunai Cepat,1400000.0
1,Tunai Cepat,1200000.0
2,Tunai Cepat,307000.0
3,Dana Tercepat,263000.0
4,Dana Tercepat,367000.0
...,...,...
95,Tunai Cepat,700000.0
96,Pinjam Uang,1600000.0
97,Wall In,1040000.0
98,Dana Tercepat,511000.0


In [5]:
my_data['loan_suc'] = my_data['message'].apply(lambda x: 1 if x == '资金已到账' else 0)
my_data['pay_suc'] = my_data['message'].apply(lambda x: 1 if x == '还款成功' else 0)
my_data['remind_due'] = my_data['message'].apply(lambda x: 1 if x == '账单已逾期N天请还款' else 0)
my_data['remind_due_sb'] = my_data['message'].apply(lambda x: 1 if x == '提醒某人还款' else 0)

1)在一个平台有多次成功放款，且还款次数
1)APP名称 2）在一个平台多次成功放款 3）最近一次放款距离现在时间差 4）放款金额 5）放款平台数 6）有多少个APP发送信息

In [6]:
frame1 = my_data.groupby(['Send to'])
my_frame = pd.DataFrame()
my_frame['cnt_message'] = frame1['Message ID'].agg('count')
my_frame['cnt_unique_app'] = frame1.agg({'app_name':pd.Series.nunique})
my_frame['cnt_loan_suc'] = frame1['loan_suc'].agg('sum')
my_frame['cnt_pay_suc'] = frame1['pay_suc'].agg('sum')
my_frame['cnt_remind_due'] = frame1['remind_due'].agg('sum')
my_frame['cnt_remind_due_sb'] = frame1['remind_due_sb'].agg('sum')
my_frame['min_cur_fund'] = frame1['cur_fund'].agg('min')
my_frame['max_cur_fund'] = frame1['cur_fund'].agg('max')
print(1)
my_frame['min_amount'] = frame1['amount'].agg('min')
my_frame['max_amount'] = frame1['amount'].agg('max')

1


In [7]:
my_data['Send to'].nunique()

310970

In [8]:
my_frame.to_csv(file_path_out + 'MGI_result_1218.csv')

In [9]:
my_frame['cnt_gap'] = my_frame['cnt_loan_suc'] - my_frame['cnt_pay_suc']

In [ ]:
1）测试放款次数>1 且   放款次数 - 成功还款次数<1;
复贷客户，且每次都还款，共计（11559人）
2）测试放款次数>0 且 放款次数 - 成功还款次数<1;
有过放款记录（含新老用户），且每次都还款 共计（26652人）
3）测试放款次数>1 且还款次数 >0
复贷客户，且有过还款记录，共计（21372人）
4）测试放款次数>0 且还款次数 >0
有过放款记录（含新老用户），且有过还款记录 共计（36465人）

In [10]:
# 1）测试放款次数>1 且   放款次数 - 成功还款次数<1. (说明基本上每次都还款)1
re1 = my_frame[(my_frame['cnt_loan_suc']>1) & (my_frame['cnt_gap']<1)]
re1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11559 entries, 6281225709 to 62895807445167
Data columns (total 11 columns):
cnt_message          11559 non-null int64
cnt_unique_app       11559 non-null int64
cnt_loan_suc         11559 non-null int64
cnt_pay_suc          11559 non-null int64
cnt_remind_due       11559 non-null int64
cnt_remind_due_sb    11559 non-null int64
min_cur_fund         11559 non-null float64
max_cur_fund         11559 non-null float64
min_amount           11559 non-null float64
max_amount           11559 non-null float64
cnt_gap              11559 non-null int64
dtypes: float64(4), int64(7)
memory usage: 1.1 MB


In [12]:
# 2）测试放款次数>0 且 放款次数 - 成功还款次数<1
re2 = my_frame[(my_frame['cnt_loan_suc']>0) & (my_frame['cnt_gap']<1)]
re2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26652 entries, 6281225709 to 62896233777916
Data columns (total 11 columns):
cnt_message          26652 non-null int64
cnt_unique_app       26652 non-null int64
cnt_loan_suc         26652 non-null int64
cnt_pay_suc          26652 non-null int64
cnt_remind_due       26652 non-null int64
cnt_remind_due_sb    26652 non-null int64
min_cur_fund         26652 non-null float64
max_cur_fund         26652 non-null float64
min_amount           26652 non-null float64
max_amount           26652 non-null float64
cnt_gap              26652 non-null int64
dtypes: float64(4), int64(7)
memory usage: 2.4 MB


In [13]:
# 3）测试放款次数>1 且还款次数 >0
re3 = my_frame[(my_frame['cnt_loan_suc']>1) & (my_frame['cnt_pay_suc']>0)]
re3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21372 entries, 6281225709 to 62895808601515
Data columns (total 11 columns):
cnt_message          21372 non-null int64
cnt_unique_app       21372 non-null int64
cnt_loan_suc         21372 non-null int64
cnt_pay_suc          21372 non-null int64
cnt_remind_due       21372 non-null int64
cnt_remind_due_sb    21372 non-null int64
min_cur_fund         21372 non-null float64
max_cur_fund         21372 non-null float64
min_amount           21372 non-null float64
max_amount           21372 non-null float64
cnt_gap              21372 non-null int64
dtypes: float64(4), int64(7)
memory usage: 2.0 MB


In [14]:
# 4）测试放款次数>0 且还款次数 >0
re4 = my_frame[(my_frame['cnt_loan_suc']>0) & (my_frame['cnt_pay_suc']>0)]
re4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36465 entries, 6281225709 to 62896233777916
Data columns (total 11 columns):
cnt_message          36465 non-null int64
cnt_unique_app       36465 non-null int64
cnt_loan_suc         36465 non-null int64
cnt_pay_suc          36465 non-null int64
cnt_remind_due       36465 non-null int64
cnt_remind_due_sb    36465 non-null int64
min_cur_fund         36465 non-null float64
max_cur_fund         36465 non-null float64
min_amount           36465 non-null float64
max_amount           36465 non-null float64
cnt_gap              36465 non-null int64
dtypes: float64(4), int64(7)
memory usage: 3.3 MB


In [8]:
print(max(my_data['amount']), min(my_data['amount']))

2960000.0 10000.0


In [13]:
pd.crosstab(my_data['amount'], my_data['amount'],margins = True).to_csv(file_path_out + 'temp.csv')

In [38]:
Counter(my_data['app_name'])

Counter({'Tunai Cepat': 423767,
         'Dana Tercepat': 532327,
         'Sumber Dana': 93261,
         'Pinjam Uang': 194539,
         'Wall In': 54983,
         'Dana Segera': 15145,
         nan: 16609,
         'KoinMu': 5,
         'Pasar Uang': 21,
         'One Hope': 264,
         'Modal Pinjaman': 10,
         'Lahan Uang': 1})

In [5]:
Counter(my_data['message'])

Counter({'还款成功': 88496,
         '展期成功': 54534,
         '还款失败请重新提交': 4287,
         '账单今日到期请还款': 276247,
         '提早或及时付款，您将有机会获得贴现利息并提高限额': 6193,
         '提醒某人还款': 309148,
         '账单已逾期N天请还款': 288754,
         'other': 48,
         '验证码': 198320,
         '资金已到账': 104905})

In [51]:
pd.crosstab(my_data['period'], my_data['message'])

message,other,展期成功,资金已到账
period,,,
0.0,0,0,1
1.0,1,0,0
6.0,0,0,4
7.0,0,769,104879
8.0,0,51160,0
9.0,0,1901,0
10.0,0,279,0
11.0,0,105,0
12.0,0,49,1


In [16]:
my_data.head(10)

,Message ID,Date/Time,From,Send to,Message Title,Message Status,app_name,amount,bank_code,message_type,deadline,remind_sb,remind_self,message,dpd_n,period,cur_fund
0,21191898194,2019-11-30,CMK,628156961290,Indosat,DELIVERED,Tunai Cepat,1400000,NaN,other,NaN,NaN,NaN,还款成功,NaN,NaN,-1
1,21191898028,2019-11-30,CMK,6281395300667,Telkomsel,DELIVERED,Tunai Cepat,1200000,NaN,other,NaN,NaN,NaN,还款成功,NaN,NaN,-1
2,21191897974,2019-11-30,CMK,6285933192866,XL,DELIVERED,Tunai Cepat,307000,NaN,other,2019-12-08,NaN,NaN,展期成功,NaN,8.0,-1
3,21191897775,2019-11-30,CMK,6282350758802,Telkomsel,DELIVERED,Dana Tercepat,263000,NaN,other,2019-12-08,NaN,NaN,展期成功,NaN,8.0,-1
4,23198282931,2019-11-30,CMK,6281553106506,Indosat,DELIVERED,Dana Tercepat,367000,NaN,other,2019-12-08,NaN,NaN,展期成功,NaN,8.0,-1
5,21191897280,2019-11-30,CMK,6289643391330,Three,DELIVERED,Tunai Cepat,1296000,NaN,other,NaN,NaN,NaN,还款成功,NaN,NaN,-1
6,21191897132,2019-11-30,CMK,6283812345665,AXIS,DELIVERED,Sumber Dana,868000,NaN,other,NaN,NaN,NaN,还款成功,NaN,NaN,-1
7,21191896873,2019-11-30,CMK,6287828592007,XL,DELIVERED,Tunai Cepat,832000,NaN,other,NaN,NaN,NaN,还款成功,NaN,NaN,-1
8,21191896634,2019-11-30,CMK,6282152348881,Telkomsel,DELIVERED,Dana Tercepat,1400000,NaN,other,NaN,NaN,NaN,还款成功,NaN,NaN,-1
9,21191896631,2019-11-30,CMK,6282335634028,Telkomsel,DELIVERED,Dana Tercepat,1440000,NaN,other,NaN,NaN,NaN,还款成功,NaN,NaN,-1
